In [44]:
using LinearAlgebra
using Plots
using PrettyTables

#funkcje pomocnicze
function  h(i, X)
    return X[i+1] - X[i]
end

function Δ(i, X, Y)
    return  (Y[i+1] - Y[i]) / h(i, X)
end

function find(x, X)
    if x < X[1]
        return 1
    end
    for (i, xᵢ) in enumerate(X)
        if x < xᵢ
            return i-1
        end
    end
    if x >= X[end]
        return length(X)-1
    end
end


find (generic function with 1 method)

## Funkcja przydzielona na zajęciach

In [3]:
k = 4
m = 1
A = -4*pi
B = 3*pi
my_func = (x) -> exp(-k*sin(m*x)) + k*sin(m*x) - 1
# my_derivate = (x) -> (exp(-k*sin(m*x))*(-k*m*cos(m*x)) + k*m*cos(m*x))
xs = -4*pi:0.01:3*pi

# plot(xs, my_func)
# savefig("plots/givenFunc")
# plot!(xs, my_derivate, lc="green")

-12.566370614359172:0.01:9.42362938564083

## Funkcja sklejana 3-go stopnia

In [12]:
# funkcje wypełniające brzegi w zalerzności od warunku brzegowego:

function clamped_boundary(A, b, x, y)
    n = length(x)
    h1 = h(1, x)
    hn = h(n-1, x)
    Δ1 = Δ(1, x, y)
    Δn = Δ(n-1, x, y)

    A[1,1] = 2*h1
    A[1,2] = h1
    b[1] = 3*(Δ1 - 2*h1*y[1])

    A[n,n-1] = hn
    A[n,n] = 2*hn
    b[n] = 3*(2*hn*y[n] - Δn)
end

function not_a_knot_boundary(A, b, x, y)
    n = length(x)
    h1 = h(1, x)
    h2 = h(2, x)
    hn1 = h(n-2, x)
    hn = h(n-1, x)
    Δ1 = Δ(1, x, y)
    Δ2 = Δ(2, x, y)
    Δn1 = Δ(n-2, x, y)
    Δn = Δ(n-1, x, y)

    A[1,1] = (h2 + h1)*hn^2
    A[1,2] = h1*h2*hn + h1^2*hn + h2^2*hn
    A[1,3] = -h1^2*hn
    b[1] = 3*((Δ2 - Δ1)*hn^2 - Δ1*h1^2 - Δ2*h2^2)

    A[n,n-2] = -hn1^2
    A[n,n-1] = h2*h1*hn1 + h2^2*hn1 + h1^2*hn1
    A[n,n] = (h2 + hn1)*h1^2
    b[n] = 3*((Δn1 - Δn)*h1^2 - Δn*hn1^2 - Δn1*h2^2)
end

function free_boundary(A, b, x, y)
    n = length(x)
    A[1,1] = 1.0
    b[1] = 0.0

    A[n,n] = 1.0
    b[n] = 0
end

function lecture_boundary(A, b, x, y)
    n = length(x)
    A[1,1] = -h(1,x)
    A[1,2] = -h(1,x)
    b[1] = h(1,x)^2 * Δ(1,x,y)^3

    A[n,n] = -h(n-1,x)
    A[n,n-1] = h(n-1,x)
    b[n] = h(n-1,x)^2 * Δ(n-3,x,y)^3
end

lecture_boundary (generic function with 1 method)

In [15]:
using Interpolations
# funkja zwracająca funkcje sklejaną fill_edges_func wypełnia pierwszy i ostatni wiersz macierzy A oraz wektora b
function cubic_spline_interpolation_2(x::AbstractVector, y::AbstractVector, fill_edges_func=free_boundary)
    n = length(x)

    A = zeros(n,n)
    b = zeros(n)
    
    fill_edges_func(A,b,x,y)
    
    for i in 2:n-1
        hip = h(i-1, x)
        hi = h(i, x)
        Δip = Δ(i-1, x, y)
        Δi = Δ(i, x, y)
        A[i,i-1] = hip
        A[i,i] = 2*(hi + hip)
        A[i,i+1] = hi
        b[i] = 3*(Δi - Δip)
    end
    
    σ = A\b

    # println("s1(x1)=$(s(1,x[1],x,y,σ)), s1(x2)=$(s(1,x[2],x,y,σ))")
    f = (xx) -> begin
        if xx < x[1] || xx > x[end]
            error("xx out of range")
        end
        
        i = find(xx,x)
        hi = h(i, x)
        bi = ((y[i+1] - y[i])/hi) - (hi*(σ[i+1] + 2σ[i]))
        ci = 3σ[i]
        di = (σ[i+1] - σ[i])/hi
        return y[i] + bi*(xx - x[i]) + ci*(xx - x[i])^2 + di*(xx - x[i])^3
    end
    itp_cubic = Interpolation.cubic_spline_interpolation(x,y)
    f(x) = itp_cubic(x)
    return f
end

x = 1:4
y = map((x)->x^2, x)
cubic_spline_interpolation(x, y)

p = cubic_spline_interpolation_2(x, y, free_boundary)
p(1.), p(1.5), p(2.)

UndefVarError: UndefVarError: h not defined

In [10]:

using Interpolations

x = 0:9
y = map((x)->x^2, 0:9)
f = cubic_spline_interpolation(x, y)

f(10)

BoundsError: BoundsError: attempt to access 10-element extrapolate(scale(interpolate(OffsetArray(::Vector{Float64}, 0:11), BSpline(Cubic(Line(OnGrid())))), (0:9,)), Throw()) with element type Float64 at index [10]

### Rysowanie wykresów

In [56]:
function drawInterpolation(X, func, interpolationFunc, title="" , accuracy=10e-4)
    minVal, maxVal = extrema(X)
    len = maxVal - minVal
    x = minVal:(len)*accuracy:maxVal
    plot(x, func, label="f(x)", width=1.5)
    plot!(x, interpolationFunc, label="interpolacja")
    plot!(X, map(func, X), seriestype=:scatter, label="węzły")
    title!(title)
end

function get_nodes_and_spline(A, B, f, n, spline_func=cubic_spline_interpolation, boundary_func=free_boundary)
    x = collect(range(A, B, length=n+1))
    y = f.(x)
    return x, spline_func(x,y,boundary_func)
end

function get_czebyshew_and_spline(A, B, f, n, spline_func=cubic_spline_interpolation, boundary_func=free_boundary)
    x = collect(map((k)-> (B-A)/2*cos(((2*k-1)/2n*pi)) + (A + B)/2, 1:n))
    sort!(x)
    y = f.(x)
    return x, spline_func(x,y,boundary_func)
end

get_czebyshew_and_spline (generic function with 3 methods)

In [73]:
f = my_func
n = 15
# X, intF = get_czebyshew_and_spline(A, B, f, n, cubic_spline_interpolation, clamped_boundary) 
# X, intF = get_czebyshew_and_spline(A, B, f, n, cubic_spline_interpolation, free_boundary) 
# X, intF = get_czebyshew_and_spline(A, B, f, n, cubic_spline_interpolation, not_a_knot_boundary) 
X, intF = get_nodes_and_spline(A, B, f, n, cubic_spline_interpolation_2, free_boundary) 
drawInterpolation(X, f, intF, "Funkcja sklejana 3-go stopnia")
# savefig("plots/SpliBonClamp")

MethodError: MethodError: no method matching cubic_spline_interpolation(::Vector{Float64}, ::Vector{Float64})
Closest candidates are:
  cubic_spline_interpolation(!Matched::AbstractRange, ::AbstractVector; bc, extrapolation_bc) at ~/.julia/packages/Interpolations/nDwIa/src/convenience-constructors.jl:12
  cubic_spline_interpolation(!Matched::Tuple{Vararg{AbstractRange, N}}, ::AbstractArray{T, N}; bc, extrapolation_bc) where {N, T} at ~/.julia/packages/Interpolations/nDwIa/src/convenience-constructors.jl:29

## Funkcja sklejana 2-go stopnia

In [41]:

function quadratic_spline_interpolation(x, y, fill_edges_func=free_boundary)
    n = length(x)

    A = zeros(n,n)
    b = zeros(n)

    fill_edges_func(A, b, x, y)

    for i in 2:n-1
        h₁ = h(i-1, x)
        h₂ = h(i, x)
        A[i,i-1] = h₁
        A[i,i] = 2*(h₁ + h₂)
        A[i,i+1] = h₂
        b[i] = 3*((y[i+1] - y[i])/h₂ - (y[i] - y[i-1])/h₁)
    end

    σ = A \ b
    function f(xx)
        if xx < x[1] || xx > x[end]
            error("xx out of range")
        end
        i = find(xx,x)
        hi = h(i, x)
        bi = (y[i+1] - y[i])/hi - hi/2 * σ[i+1] - hi/2 * σ[i]
        ci = 1/(2*hi) * (y[i+1] - y[i] - hi * (σ[i+1] - σ[i]))
        return y[i] + bi * (xx - x[i]) + ci * (xx - x[i])^2
    end
    return f
end

x = [0, 1, 2, 3, 4]
y = [1, 3, 2, 4, 1]

p = quadratic_spline_interpolation(x, y, free_boundary)
p(1.), p(1.5), p(2.)

UndefVarError: UndefVarError: h not defined

In [ ]:
# funkcje pomocnicze

function  h(i, X)
    return X[i+1] - X[i]
end

function Δ(i, X, Y)
    return  (Y[i+1] - Y[i]) / h(i, X)
end

function find(x, X)
    if x < X[1]
        return 1
    end
    for i in 1:length(X)-1
        if x < X[i+1]
            return i
        end
    end
    return length(X)-1
end

function free_boundary(A, b, x, y)
    n = length(x)
    A[1,1] = 1.0
    b[1] = 0.0

    A[n,n] = 1.0
    b[n] = 0
end

function quadratic_spline_interpolation(x, y, fill_edges_func=free_boundary)
    n = length(x)

    A = zeros(n,n)
    b = zeros(n)

    fill_edges_func(A, b, x, y)

    for i in 2:n-1
        h₁ = h(i-1, x)
        h₂ = h(i, x)
        A[i,i-1] = h₁
        A[i,i] = 2*(h₁ + h₂)
        A[i,i+1] = h₂
        b[i] = 3*((y[i+1] - y[i])/h₂ - (y[i] - y[i-1])/h₁)
    end

    σ = A \ b

    function f(xx)
        if xx < x[1] || xx > x[end]
            error("xx out of range")
        end
        i = find(xx,x)
        hi = h(i, x)
        bi = (y[i+1] - y[i])/hi - (hi/2) * (σ[i+1] + 2σ[i])
        ci = (σ[i+1] - σ[i]) / (6hi)
        return y[i] + bi * (xx - x[i]) + ci * (xx - x[i])^2
    end

    return f
end

x = [0, 1, 2, 3, 4]
y = [1, 3, 2, 4, 1]

p = quadratic_spline_interpolation(x, y, free_boundary)
p(1.), p(1.5), p(2.)

In [ ]:
f = my_func
n = 5
X, intF = get_nodes_and_spline(A, B, f, n, quadratic_spline_interpolation, free_boundary) 
drawInterpolation(X, f, intF, "Funkcja sklejana 2-go stopnia")
# savefig("plots/ROn$n")

## spline interpolation

In [37]:
using Interpolations, Plots

# Lower and higher bound of interval
a = A
b = B
# Interval definition
x = range(a-0.5, b+0.5, length=25)
# This can be any sort of array data, as long as
# length(x) == length(y)
y = @. my_func(x) # Function application by broadcasting
# Interpolations
itp_cubic = cubic_spline_interpolation(x,y)
itp_quadratic = interpolate(y, BSpline(Quadratic(Flat(OnGrid()))))

# Interpolation functions
f_cubic(x) = itp_cubic(x)
f_quadratic(x) = itp_quadratic(x)
# # Plots
# width, height = 1500, 800 # not strictly necessary
x_new = a:0.1:b # smoother interval, necessary for cubic spline

scatter(x, y, markersize=3,label="Data points")
plot!(my_func, x_new,label="my_func")
plot!(f_cubic, x_new, linestyle=:dash, label="Cubic Spline interpolation")
plot!(f_quadratic, x_new, linestyle=:dot, w=3, label="Quadratic B-spline interpolation")
# plot!(size = (width, height))
# plot!(legend = :bottomleft)

BoundsError: BoundsError: attempt to access 25-element interpolate(OffsetArray(::Vector{Float64}, 0:26), BSpline(Quadratic(Flat(OnGrid())))) with element type Float64 at index [-12.566370614359172]

In [42]:

using Interpolations, Plots

# Lower and higher bound of interval
a = A
b = B
# Interval definition
x = range(a, b, length=16)
# This can be any sort of array data, as long as
# length(x) == length(y)
y = @. my_func(x) # Function application by broadcasting

# Interpolations
itp_linear = linear_interpolation(x, y)
itp_cubic = cubic_spline_interpolation(x,y, bc = Free(OnGrid()))
# itp_quadratic = quadratic_interpolation(x, y)

# if a <= -1 && b >= 1
#     itp_quadratic = interpolate(y, BSpline(Quadratic(Flat(OnGrid()))))
# else
#     itp_quadratic = quadratic_interpolation(x, y)
# end

# Interpolation functions
f_linear(x) = itp_linear(x)
f_cubic(x) = itp_cubic(x)
# f_quadratic(x) = itp_quadratic(x)
# Plots
print(typeof(f_cubic))
width, height = 1500, 800 # not strictly necessary
x_new = a:0.1:b # smoother interval, necessary for cubic spline

plot(my_func, x_new, label="funkcja pierwotna")
plot!(f_cubic, x_new, linestyle=:dash, label="interpolacja")
scatter!(x, y,label="punkty")
# plot!(f_quadratic, x_new, linestyle=:dot, w=3, label="Quadratic interpolation")
# plot!(size = (width, height))
plot!(legend = :bottomleft)
savefig("plots/Spli16")

typeof(f_cubic)

"/home/ciastek/Dokumenty/studia/MoWNiT/MOwNiT/lab3/plots/Spli16.png"